# OPC UA a SQLite (Nuevo Esquema - 12 Variables)

Este notebook se conecta al PLC mediante `asyncua`, busca las 12 variables principales del MES, y las inserta periódicamente en la base de datos `datos.db`.

In [ ]:
import asyncio
import sqlite3
import logging
import os
from datetime import datetime
from asyncua import Client

# Configuraciones OPC UA
ENDPOINT = "opc.tcp://192.168.0.20:4840"
DB_PATH = "../database/datos.db"
DB_NAME = "MES"

# Nombres de las 12 variables a buscar (Pascal_Case)
VAR_NAMES = [
    "Machine_State", "Heartbeat", "Target_Speed", "Total_Parts_Produced",
    "Parts_OK", "Parts_NOK", "Last_Cycle_Time", "Availability",
    "Perfomance", "Quality", "Initial_Timestamp", "Final_Timestamp"
]

logging.basicConfig(level=logging.INFO)
_logger = logging.getLogger('opcua_sqlite')

### 1. Inicializar Base de Datos
Aseguramos que existe el directorio y creamos la tabla con los 12 campos nuevos.

In [ ]:
def init_db():
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Creamos la tabla adaptada al nuevo esquema (usando los nombres oficiales)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mes_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            db_timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
            Machine_State INTEGER,
            Heartbeat INTEGER,
            Target_Speed REAL,
            Total_Parts_Produced INTEGER,
            Parts_OK INTEGER,
            Parts_NOK INTEGER,
            Last_Cycle_Time REAL,
            Availability REAL,
            Perfomance REAL,
            Quality REAL,
            Initial_Timestamp TEXT,
            Final_Timestamp TEXT
        )
    ''')
    conn.commit()
    return conn

conn = init_db()
_logger.info(f"Base de datos conectada en {DB_PATH}")

### 2. Helper para buscar nodos
Función recursiva para encontrar nodos dentro de la carpeta Objects por su BrowseName.

In [ ]:
async def find_node_by_name(client, db_name, var_name, max_depth=5):
    objects = client.nodes.objects
    
    async def walk(node, depth):
        if depth > max_depth: return None
        try: children = await node.get_children()
        except: return None
        
        for child in children:
            try: bname = (await child.read_browse_name()).Name
            except: continue
            
            if bname == db_name:
                db_children = await child.get_children()
                for var in db_children:
                    if (await var.read_browse_name()).Name == var_name:
                        return var
            
            found = await walk(child, depth + 1)
            if found: return found
        return None
    return await walk(objects, 0)

### 3. Bucle Principal Asíncrono OPC UA
Conectamos, localizamos los 12 nodos e insertamos continuamente.

In [ ]:
async def main_loop():
    _logger.info(f"Conectando a {ENDPOINT}...")
    async with Client(url=ENDPOINT) as client:
        _logger.info("Conectado a OPC UA.")
        
        # 1. Resolver los 12 nodos OPC UA
        _logger.info("Resolviendo Nodos...")
        nodes = {}
        for vname in VAR_NAMES:
            node = await find_node_by_name(client, DB_NAME, vname)
            if getattr(node, 'nodeid', None):
                nodes[vname] = node
                # _logger.info(f"Encontrado {vname}: {node.nodeid}")
            else:
                _logger.error(f"No se encontró la variable: {vname}")
        
        if len(nodes) < len(VAR_NAMES):
            _logger.error("Faltan nodos. Deteniendo el script para revisar la nomenclatura.")
            return
            
        _logger.info("Todos los nodos encontrados. --- INICIANDO LECTURA E INSERCIÓN ---")
        
        cursor = conn.cursor()
        
        try:
            while True:
                # Leer todos los valores en un dict comprehension
                values = {name: await node.read_value() for name, node in nodes.items()}
                
                # Preparar sentencia SQL (12 interrogantes para las 12 vars)
                placeholders = ", ".join(["?"] * len(VAR_NAMES))
                columns = ", ".join(VAR_NAMES)
                
                query = f"INSERT INTO mes_data ({columns}) VALUES ({placeholders})"
                
                # Tupla ordenada con los valores leídos para insertar en SQLite
                data_tuple = tuple(values[v] for v in VAR_NAMES)
                
                cursor.execute(query, data_tuple)
                conn.commit()
                
                # Print amigable (mostrando solo 4 métricas clave en consola para no ensuciar)
                print(f"[SQL Insert] HB:{values['Heartbeat']} | ST:{values['Machine_State']} | Spd:{values['Target_Speed']:.1f} | Prod:{values['Total_Parts_Produced']} | OEE:{values['Perfomance']*100:.1f}%")
                
                await asyncio.sleep(1.0) # Ciclo de captura de 1s
                
        except asyncio.CancelledError:
            _logger.info("Lectura detenida por el usuario.")
            
try:
    task = asyncio.create_task(main_loop())
    await task
except KeyboardInterrupt:
    task.cancel()

### 4. Consultar resultados con Pandas

In [ ]:
import pandas as pd

# Leemos la tabla entera de SQLite y la ponemos en un DataFrame
df = pd.read_sql_query("SELECT * FROM mes_data ORDER BY id DESC LIMIT 10", conn)

print("Últimos 10 registros en SQLite:")
display(df)